In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext

C:\ANACONDA\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
   city = 'orlando/' #*****change this city to what you want*****
   url = 'https://www.zillow.com/homes/for_sale/'+city    
   r = s.get(url, headers=req_headers)

In [5]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
}
     


In [6]:
base_url = "https://www.zillow.com/homes/for_sale/"

In [8]:
urls = []

city = 'orlando'
url1 = base_url +city+'/'
urls.append(url1)
for i in range(2,21):
    dom = base_url + city +'/'+str(i)+'_p/'
    urls.append(dom)

In [9]:
print(urls)

['https://www.zillow.com/homes/for_sale/orlando/', 'https://www.zillow.com/homes/for_sale/orlando/2_p/', 'https://www.zillow.com/homes/for_sale/orlando/3_p/', 'https://www.zillow.com/homes/for_sale/orlando/4_p/', 'https://www.zillow.com/homes/for_sale/orlando/5_p/', 'https://www.zillow.com/homes/for_sale/orlando/6_p/', 'https://www.zillow.com/homes/for_sale/orlando/7_p/', 'https://www.zillow.com/homes/for_sale/orlando/8_p/', 'https://www.zillow.com/homes/for_sale/orlando/9_p/', 'https://www.zillow.com/homes/for_sale/orlando/10_p/', 'https://www.zillow.com/homes/for_sale/orlando/11_p/', 'https://www.zillow.com/homes/for_sale/orlando/12_p/', 'https://www.zillow.com/homes/for_sale/orlando/13_p/', 'https://www.zillow.com/homes/for_sale/orlando/14_p/', 'https://www.zillow.com/homes/for_sale/orlando/15_p/', 'https://www.zillow.com/homes/for_sale/orlando/16_p/', 'https://www.zillow.com/homes/for_sale/orlando/17_p/', 'https://www.zillow.com/homes/for_sale/orlando/18_p/', 'https://www.zillow.co

In [18]:
# Function to get data from urls and parse it
def soups(data):
    with requests.Session() as s:
        r = s.get(data, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [19]:
# Call soup function and store output in a list
lst = []

for url in urls:
    htmls = soups(url)
    lst.append(htmls)
print(len(lst))

20


In [20]:
df = pd.DataFrame()

In [21]:
def my_soup(data):
    
    address = data.find_all(class_= 'list-card-addr')
    price = list(data.find_all(class_='list-card-price'))
    beds = list(data.find_all("ul", class_="list-card-details"))
    last_updated = data.find_all('div', {'class': 'list-card-top'})
    
    #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds
    df['last_updated'] = last_updated
        
    return df.copy()

In [14]:
zillow_df = pd.DataFrame()

# Get our data from each url and put it into a list of dataframes
df_list = []
for soup in lst:
    new_df = my_soup(soup)
    df_list.append(new_df)
    
# Combine the list of datasets into our new dataframe
zillow_df = pd.concat(df_list)
zillow_df.reset_index(inplace=True)
zillow_df = zillow_df.drop('index', axis=1)

ValueError: Length of values (10) does not match length of index (9)

In [ ]:
print(zillow_df.shape)
zillow_df.head()

In [ ]:
print(zillow_df.dtypes)

In [ ]:
zillow_df = zillow_df.applymap(str)

In [ ]:
zillow_df = zillow_df.applymap(lambda x: re.sub('<[^<]+?>', '',x))
zillow_df.head()

In [ ]:
zillow_df['address'].value_counts()

In [ ]:
zillow_df[['beds', 'home_type']] = zillow_df.beds.str.split('-',n=1, expand=True)
zillow_df.head()

In [ ]:
# separate beds column into bed, bath, and sq_feet
zillow_df[['beds', 'baths', 'sqft']] = zillow_df.beds.str.split(' ',n=2, expand=True)
zillow_df['sqft'] = zillow_df.sqft.str.replace(",", "")

# extract only the digits from the columns
zillow_df['beds'] = zillow_df.beds.str.extract('(\d+)')
zillow_df['baths'] = zillow_df.baths.str.extract('(\d+)')
zillow_df['sqft'] = zillow_df.sqft.str.extract('(\d+)')

# convert columns to float
zillow_df['beds'] = zillow_df['beds'].astype('float')
zillow_df['baths'] = zillow_df['baths'].astype('float')
zillow_df['sqft'] = zillow_df['sqft'].astype('float')

zillow_df.head()

In [ ]:
zillow_df.info()

In [ ]:
zillow_df.to_csv("Zillow_Home_Listings.csv", index=False)